In [ ]:
from transformers import AutoTokenizer, AutoModel

# KoBERT tokenizer 로드
tokenizer = AutoTokenizer.from_pretrained("monologg/kobert", trust_remote_code=True)
bert = AutoModel.from_pretrained("monologg/kobert")

# 1024 길이의 텍스트 예제
text = "여기에 1024 길이의 텍스트를 넣으세요. " * 50  # 텍스트 반복으로 1024 길이 맞춤

# 텍스트 토큰화
tokenized = tokenizer(
    text,
    padding="max_length",  # 최대 길이로 패딩
    truncation=True,       # 초과된 길이는 잘라냄
    max_length=1024,       # 최대 길이 지정
    return_tensors="pt"    # PyTorch 텐서 형식으로 반환
)

# 출력 확인
print(tokenized["input_ids"])

article_output = bert(
                    tokenized["input_ids"],
                    attention_mask=tokenized["attention_mask"],
                    return_dict=True
                )
    
print(article_output)

tensor([[   2, 3302,  533,  ...,    1,    1,    1]])


RuntimeError: The expanded size of the tensor (1024) must match the existing size (512) at non-singleton dimension 1.  Target sizes: [1, 1024].  Tensor sizes: [1, 512]

In [ ]:
import torch

text = "여기에 1024 길이의 텍스트를 넣으세요. " * 50 

texts = [text[i:i + 512] for i in range(0, len(text), 512)]

outputs = []
for part in texts:
    tokenized = tokenizer(
        part,
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )
    
    output = bert(
        input_ids=tokenized["input_ids"],
        attention_mask=tokenized["attention_mask"],
        return_dict=True
    )
    outputs.append(output.last_hidden_state)

# 결과 병합
merged_output = torch.cat(outputs, dim=1)

final_length = 1024
if merged_output.size(1) < final_length:
    padding_length = final_length - merged_output.size(1)
    pad = torch.zeros((merged_output.size(0), padding_length, merged_output.size(2)))
    final_output = torch.cat([merged_output, pad], dim=1)
else:
    final_output = merged_output[:, :final_length, :]  # 첫 1024 토큰만 사용

print(final_output.shape)

torch.Size([1, 1024, 768])
torch.Size([1, 1536, 768])
